<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS17 Template - an Overview</span> </p>

# Introduction

Systemorph's IFRS 17 Calculation Engine is a tool and a solution to conduct all the essential IFRS 17 calculations based on some key input: modeled future **cashflows** and actual amounts for **groups of insurance contracts** and other relevant parameters (Yield Curve, FX rates, etc...). The output is a set of calculated results (Contractual Service Margin, Loss Component, Financial Performance, etc...) that are needed for IFRS 17 reporting, analysis, accounting, steering and other management information.

Systemorph's Full IFRS 17 Template provides a complete example of input data and uses the Calculation Engine for the production of IFRS 17 results.  Users can adapt this template to their own needs by changing the input data and calculating their own results.

This notebook provides an **overview** of Systemorph's Full IFRS 17 Template with brief introductions to all the main steps.

<a id='ifrs17-intro'></a>
# IFRS 17

IFRS 17 is a new accounting standard for insurance contracts. The previous standard IFRS 4, which is based on nominal accounting, is replaced by the economic accounting standard IFRS 17.

Some key points are repeated here briefly.

IFRS 17 deals with insurance contracts which are on the liability side of the balance sheet.  Other accounting standards deal with other parts of the business, primarily IFRS 9 for invested assets.

Insurance policies need to be allocated to homogeneous groups of insurance contracts (GICs) according to high-level principles established by the IFRS 17 standard, such as contract type, line of business, annual cohort etc. Insurers have some degree of freedom to optmize those GICs, as long as they respect the high-level principles.

Insurance contracts generate different types of cashflows. For example, **Amount Type** is a dimension refering to premiums, claims, expenses of different kinds, etc. for these cashflows. The Calculation Engine presented here expects input files of all the projected cashflows for each GIC, for different regular time points (quarterly or monthly, ...) and corresponding actual amounts observed for a given period. 

Each cashflow is characterized by all the necessary attributes: business unit, start time, amount type, novelty of business etc. and projected cashflow amounts in a regular time grid until the final run-off of all contracts.

If the entered cashflows are nominal (the usual case), the the present value (PV) of each cashflow is computed using proper yield curves for discounting.

A central element of IFRS 17 is the Analysis of Change (AoC).  The present value of the business (future cashflows) changes from the beginning to the end of the accounting period due to different effects, each of them constituting a modeled AoC step.

In IFRS 17, the value of future profitable business has a component called the Contractual Service Margin (CSM), defined at GIC level. The profit measured as CSM is recognized slowly over time rather than immediately in financial performance statements. On the other hand, a GIC can be onerous and produce a Loss Component (LC) instead, which needs to be recognized immediately.

Several reports on the valuation of cashflows, balance sheet and financial performance can be produced, each dealing with different aspects, such as
 - Present Value of Future Cash Flows (PVFCF)
 - Contractual Service Margin (CSM) and Loss Component (LC)
 - Liability for Remaining Coverage (LRC)
 - Liability for Incurred Claims (LIC)
 - Financial Performance

# Data input



<a id='data-input-dim-nodes'></a>
## Data input: Reporting nodes, dimensions

See folders *Files* (Dimensions) and *Files/ReportingNodes* (ReportingNodes).
- **ReportingNodes**: List of business units, often per country, with system name and display name, currency and "G" for the whole group
$$
$$
- **Dimensions**: The many dimensions required to characterize all of the data and results, with system names and display names.  This information is quite stable.

<a id='data-input-data-nodes'></a>
## Data input: Groups of insurance and reinsurance contracts

See folder Files/DataNodes.  The information is about data nodes, that is Groups of Insurance Contracts (GICs) and Groups of Reinsurance Contracts (GRICs) and portfolios of these.
- **DataNodes**: List of all GICs and GRICs, each with several attributes and the portfolios to which they belong; an additional attribute of GRICs being the reinsurance partner
$$
$$
- **DataNodeParameters** *\_<reportingNode\>* *\_<year\>* *\_<month\>*: List of those GICs and GRICs that are actually used for the given reporting node/year/month combination with few attributes; if needed, more such files may be added for other reporting node/year/month combinations
$$
$$
- **DataNodeStates** *\_<reportingNode\>* *\_<year\>* *\_<month\>*: Switching data nodes on and off, the normal state is ("Active") for all the GICs and GRICs of the reporting node/year/month combination; if needed, more such files may be added for other reporting node/year/month combinations

<a id='data-input-parameters'></a>
## Data input: Parameters from financial markets

See folder Files/Parameters.
- **YieldCurve**: List of all yield curves to be used, for different currencies and as-of dates (year/month), using a yearly grid of interest rate maturities extending as far as needed
$$
$$
- **ExchangeRate**: List of exchange rates to be used, for different currencies and as-of dates (year/month), with a type (Spot/Average); defined as the value of 1 foreign unit expressed in the group currency of the insurance company 
$$
$$
- **CreditDefaultRate**: For relevant year/month combinations, each with a full list of all credit ratings in S&P notation with their default rate (= probabilty of default within 1 year)
$$
$$
- **PartnerRating**: For each reinsurance partner of a relevant GRIC, the rating in S&P notation with the as-of date (year/month)

<a id='data-input-transactional'></a>
## Data input: Groups of insurance and reinsurance contracts

See folder Files/TransactionalData.
- **Actuals** *\_<reportingNode\>* *\_<year\>* *\_<month\>*: List of all actual amounts for those GICs and GRICs that are used by the given reporting node/year/month combination, for different amount types, AoC types and other attributes; we need at least two such files for the two dates of the Analysis of Change
$$
$$
- **Openings** *\_<reportingNode\>* *\_<year\>* *\_<month\>*: List of the opening amounts for some GICs that are used by the given reporting node.  The year/month combination corresponds to the beginning of the planned Analysis of Change period, so only one such file is required.  The entries are by EstimateType and AmountType.
$$
$$
- **NominalCashflows** *\_<reportingNode\>* *\_<year\>* *\_<month\>*: List of all projected cash flows for those GICs and GRICs that are used by the given reporting node/year/month combination, for different amount types, AoC types, novelty types and other attributes, where the cash flow amounts are given in a regular sequence extending as far as needed; we need at least two such files for the two dates of the Analysis of Change

The structure of the [Actuals](./InputFormatDescription#actual), [Openings](./InputFormatDescription#opening) and [NominalCashflows](./InputFormatDescription#cashflow) files is explained in a special [notebook](./InputFormatDescription).

The less usual option of having discounted instead of nominal cash flows in the input is supported, but not a subject of this notebook in its current state.

# Initialization

With **Initialization** we refer to an automatic import of data.
Import of dimensions is a necessary step which occurs only once in a production application. 
In the context of the present **Full IFRS 17 Template** project other data, such as data nodes and transactional data, are imported through one of the following notebooks:

- [InitSystemorphToDatabase](./Initialization/InitSystemorphToDatabase): where a physical database is used as target data store (database)
$$
$$
- [InitSystemorphToMemory](./Initialization/InitSystemorphToMemory): where memory of the server hosting the application is used as target data store (in-memory). Note that, this memory is lost when connection to the server is interrupted. 

In the **Full IFRS 17 Template** project we always rely on the in-memory set up in order to be independet from a physical database which might not be available to every user. 


<a id='reports'></a>
# Reports

All the reports are produced by running a notebook such as **[Reports](./Report/Reports#report-production)**.